In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hmmlearn import hmm
from sklearn.metrics import confusion_matrix
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from dotenv import load_dotenv
import os
import warnings
from scipy.sparse import vstack
from scipy.sparse import save_npz
from scipy.sparse import load_npz
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import csr_matrix
import spacy
import re
from tqdm import tqdm
from collections import Counter
from openai import OpenAI

load_dotenv()


True

In [2]:
def aggressive_cleaning(text):
    """
    Cleans the text by handling NaN values and 'nan' strings, normalizing dashes, lowercasing,
    and removing non-alphabetic characters except spaces.
    """
    text = str(text)
    if text.lower() == 'nan' or pd.isna(text):
        return ''
    else:
        text = text.replace('-', ' ').lower().strip()
        text = re.sub(r'[^a-z\s]', '', text)
        return text

In [3]:
df = pd.read_csv('archive/utterances-2sp.csv')
df = df.dropna(subset=['utterance'])
df['utterance'] = df['utterance'].apply(aggressive_cleaning)

In [19]:
client = OpenAI()
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [21]:
def filter_episodes(df, host_id=None):
    df_filtered = df[df['host_id'] != -1]
    if host_id is None:
        top_host = df_filtered.groupby('host_id')['episode'].nunique().idxmax()
    else:
        top_host = host_id
    top_host_episodes = df_filtered[df_filtered['host_id'] == top_host]['episode'].unique()
    df_top_host_all_utterances = df[df['episode'].isin(top_host_episodes)]
    utterance_counts = df_top_host_all_utterances.groupby('episode')['utterance'].count()
    episodes_over_30 = utterance_counts[utterance_counts > 30].index
    df_top_host_over_30 = df_top_host_all_utterances[df_top_host_all_utterances['episode'].isin(episodes_over_30)]
    return df_top_host_over_30.reset_index(drop=True)

filtered_df = filter_episodes(df, host_id=1)
len(filtered_df)

46765